In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# relative imports
from uncertainty_aware_diagnosis import split_and_save_csv, ICD10data

# absolute imports
import polars as pl

# paths
train_csv = "./data/lbz-train.csv"
val_csv = "./data/lbz-val.csv"
test_csv = "./data/lbz-test.csv"
input_csv = "./data/opnamen.csv"

In [ ]:
# inspect data
opnamen = pl.read_csv(input_csv)
print(opnamen.shape)
opnamen.head()

# # check missing values
# for col in opnamen.columns:
#     print(f'{col} contains {opnamen["DBC_specialty_code"].is_null().sum()} missing values')

### preprocessing
- drop NaN values
- removed 3rd gender (n=1)
- drop classes with less than 25 samples

In [ ]:
target = "icd10_main_code"
categorical = [
    "zorginstellingnaam",
    "gender",
    "clinical_specialty",
    "DBC_specialty_code",
    "DBC_diagnosis_code",
    "icd10_subtraject_code",
]
numerical = ["age"]

data = ICD10data(
    csv_path=input_csv,
    numerical=numerical,
    categorical=categorical,
    high_card=[],
    target=target,
    dropna=True,
    use_embedding=False,
)

In [ ]:
# split data into train, val, test
split_and_save_csv(
    input_csv=input_csv,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=test_csv,
    train_frac=0.7,
    val_frac=0.15,
    test_frac=0.15,
    stratify_col=target,  # preserve class balance
    min_class_count=25,
    random_state=42,
)